In [1]:
!nvidia-smi

Sun Jul 14 23:57:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Install specific versions of the following Python packages:
# 1. accelerate==0.21.0: A package for easily managing multi-GPU and distributed training.
# 2. peft==0.4.0: A package for Parameter Efficient Fine-Tuning of pre-trained models.
# 3. bitsandbytes==0.40.2: A package providing 8-bit optimizers and quantization techniques to save memory.
# 4. transformers==4.31.0: The Hugging Face Transformers library for natural language processing (NLP) models.
# 5. trl==0.4.7: A package for Training, Reinforcement Learning, and Language modeling.
# The -q flag is used to suppress the output, making the installation process quieter and cleaner.
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Import the torch module, a deep learning framework
import torch

# Import the os module for interacting with the operating system
import os

# Import the load_dataset function from the datasets library to load datasets
from datasets import load_dataset

# Import LoraConfig, get_peft_model, and PeftModel from the peft library
# LoraConfig: Configuration for Low-Rank Adaptation (LoRA) for fine-tuning models
# get_peft_model: Function to get a PEFT (Parameter Efficient Fine-Tuning) model
# PeftModel: Class for PEFT models
from peft import LoraConfig, get_peft_model, PeftModel

# Import SFTTrainer from the trl library for supervised fine-tuning of models
from trl import SFTTrainer

In [4]:
# Import various components from the transformers library for model and tokenizer management
from transformers import (
    AutoModelForCausalLM,  # For loading pre-trained causal language models
    AutoTokenizer,        # For loading pre-trained tokenizers
    BitsAndBytesConfig,   # Configuration for bits and bytes optimizations
    HfArgumentParser,     # For parsing command-line arguments specific to Hugging Face models
    TrainingArguments,    # For specifying training configurations
    pipeline,             # For creating pipelines for inference tasks
    logging               # For managing logging within the transformers library
)

In [5]:
model_name= "NousResearch/Llama-2-7b-chat-hf"

In [6]:
# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

In [7]:
# Fine-tuned model name
new_model = "Llama-2-7b-chat-new-finetune"

In [8]:
from datasets import load_dataset

In [9]:
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [11]:
dataset[0]

{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}

In [12]:
# LoRA parameters
##################

# LoRA attention dimension
lora_r = 64


# Alpha parameter for LoRA scaling
lora_alpha = 16


# Dropout probability for LoRA layers
lora_dropout = 0.1


##############################
# bitsandbytes parameters
###############################

# Activate 4-bit precision base model loading
use_4bit = True


# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"


# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"


# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


##################################
# TrainingArguments parameters
##################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"


# Number of training epochs
num_train_epochs = 1


# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False


# Batch size per GPU for training
per_device_train_batch_size = 4



# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25


#######################
# SFT parameters
#######################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [13]:
# Load tokenizer and model with LoRA configuration

# Define the compute data type using the bitsandbytes (bnb) library for 4-bit computations
# 'bnb_4bit_compute_dtype' should be a string representing the desired torch data type (e.g., 'float16', 'float32')
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [14]:
# Create a BitsAndBytes configuration for managing 4-bit quantization and other settings
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,                        # Boolean to determine if the model should be loaded in 4-bit precision
    bnb_4bit_quant_type=bnb_4bit_quant_type,      # Type of 4-bit quantization to use (e.g., 'dynamic', 'static')
    bnb_4bit_compute_dtype=compute_dtype,         # Data type to be used for computation (e.g., torch.float16, torch.float32)
    bnb_4bit_use_double_quant=use_nested_quant,   # Boolean to determine if double quantization (nested quantization) should be used
)

In [15]:
#This function returns a tuple representing the major and minor CUDA compute capability of the currently selected GPU.
torch.cuda.get_device_capability()

(7, 5)

In [16]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    # Get the CUDA compute capability of the current GPU device
    major, _ = torch.cuda.get_device_capability()

    # Check if the major version of the compute capability is 8 or higher
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Example values for the variables used in the condition (for context):
# compute_dtype = torch.float16  # Example: Set to float16 for computation
# use_4bit = True  # Example: Set to True to enable 4-bit precision

# Explanation:
# - The condition `compute_dtype == torch.float16 and use_4bit` ensures that the code block
#   only executes if the compute data type is float16 and 4-bit quantization is enabled.
# - `torch.cuda.get_device_capability()` retrieves the CUDA compute capability of the current GPU.
# - `major` represents the major version of the compute capability.
# - If `major >= 8`, it indicates that the GPU supports bfloat16 (e.g., NVIDIA A100 GPUs),
#   and a message is printed to inform the user that they can accelerate training with bf16=True.

In [17]:
# Load a pre-trained causal language model with quantization and device mapping configurations
model = AutoModelForCausalLM.from_pretrained(
    model_name,                   # The name or path of the pre-trained model
    quantization_config=bnb_config,  # Configuration for quantization using BitsAndBytesConfig
    device_map=device_map         # A mapping of model components to devices
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [19]:
# Modify the configuration of the loaded model

# Disable the use of cache during model inference and training
model.config.use_cache = False

# Set the pretraining tensor parallelism (TP) factor to 1
model.config.pretraining_tp = 1

# Explanation:
# - model.config.use_cache = False:
#   This setting disables the use of the key-value cache during model inference and training.
#   Disabling the cache can reduce memory usage at the cost of slower inference speed,
#   particularly for autoregressive models.

# - model.config.pretraining_tp = 1:
#   This setting configures the tensor parallelism factor used during pretraining.
#   A value of 1 indicates no tensor parallelism, meaning the model will not split tensors
#   across multiple devices or processes. Adjusting this setting can help optimize training
#   and inference performance based on available hardware and specific model requirements.

In [20]:
# Load a pre-trained tokenizer and configure its settings

# Load the tokenizer for the specified model, allowing the use of remote code if necessary
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the padding token to be the same as the end-of-sequence (eos) token
tokenizer.pad_token = tokenizer.eos_token

# Set the padding side to 'right' to address issues with half-precision (fp16) training
tokenizer.padding_side = "right"  # Fixes weird overflow issue with fp16 training

# Explanation:
# - tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True):
#   This line loads the tokenizer for the specified pre-trained model. The 'trust_remote_code=True'
#   parameter allows the use of custom code from the model's repository, which might be necessary
#   for some models that have special tokenization logic.

# - tokenizer.pad_token = tokenizer.eos_token:
#   This line sets the padding token of the tokenizer to be the same as the end-of-sequence (eos) token.
#   This is often done to ensure that padding tokens do not interfere with the model's understanding
#   of the input sequences, especially in sequence generation tasks.

# - tokenizer.padding_side = "right":
#   This line sets the padding side to 'right', which means that padding tokens will be added to the
#   right (end) of the input sequences. This is particularly useful in addressing overflow issues
#   that can occur during half-precision (fp16) training, ensuring more stable and efficient training.

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [21]:
# Create a configuration for Low-Rank Adaptation (LoRA)
peft_config = LoraConfig(
    lora_alpha=lora_alpha,         # Scaling factor for the LoRA adaptation
    lora_dropout=lora_dropout,     # Dropout rate applied to the LoRA layers
    r=lora_r,                      # Rank of the low-rank decomposition
    bias="none",                   # Specifies how biases should be treated (none in this case)
    task_type="CAUSAL_LM",         # Type of task the model is being fine-tuned for (Causal Language Modeling)
)

# Explanation:
# - lora_alpha: A scaling factor that adjusts the impact of the LoRA adaptation. Higher values increase the effect.
# - lora_dropout: The dropout rate applied to the LoRA layers to prevent overfitting during training. A value between 0 and 1.
# - r: The rank of the low-rank decomposition, which determines the number of additional trainable parameters introduced by LoRA. Lower values result in fewer parameters.
# - bias="none": Specifies that no additional bias parameters should be used in the LoRA layers.
# - task_type="CAUSAL_LM": Indicates that the model is being fine-tuned for a Causal Language Modeling task, which involves predicting the next token in a sequence.

In [22]:
# Set up the training arguments for model fine-tuning
training_arguments = TrainingArguments(
    output_dir=output_dir,                             # Directory to save the model checkpoints and logs
    num_train_epochs=num_train_epochs,                 # Number of training epochs
    per_device_train_batch_size=per_device_train_batch_size, # Batch size per device during training
    gradient_accumulation_steps=gradient_accumulation_steps, # Number of steps to accumulate gradients before updating model parameters
    optim=optim,                                       # Optimizer to use (e.g., 'adamw')
    save_steps=save_steps,                             # Number of steps between model checkpoint saves
    logging_steps=logging_steps,                       # Number of steps between logging training metrics
    learning_rate=learning_rate,                       # Initial learning rate
    weight_decay=weight_decay,                         # Weight decay to apply (if any)
    fp16=fp16,                                         # Enable mixed precision training with float16
    bf16=bf16,                                         # Enable mixed precision training with bfloat16
    max_grad_norm=max_grad_norm,                       # Maximum gradient norm for gradient clipping
    max_steps=max_steps,                               # Maximum number of training steps (overrides num_train_epochs if set)
    warmup_ratio=warmup_ratio,                         # Ratio of total steps used for learning rate warmup
    group_by_length=group_by_length,                   # Group sequences of similar length together for efficient training
    lr_scheduler_type=lr_scheduler_type,               # Learning rate scheduler type (e.g., 'linear', 'cosine')
    report_to="tensorboard"                            # Reporting tool for logging (e.g., 'tensorboard', 'wandb')
)

In [23]:
# Set up the SFTTrainer for supervised fine-tuning of the model
trainer = SFTTrainer(
    model=model,                             # The pre-trained model to fine-tune
    train_dataset=dataset,                   # The training dataset
    peft_config=peft_config,                 # Low-Rank Adaptation (LoRA) configuration
    dataset_text_field="text",               # The field in the dataset that contains the text data
    max_seq_length=max_seq_length,           # The maximum sequence length for the input data
    tokenizer=tokenizer,                     # The tokenizer to use for processing the text data
    args=training_arguments,                 # The training arguments configuration
    packing=packing,                         # Whether to pack sequences to utilize maximum length efficiently
)

# Example definitions for the variables used in the SFTTrainer (for context):
# model: The pre-trained model loaded earlier
# train_dataset: The dataset used for training
# peft_config: The LoRA configuration created earlier
# max_seq_length: The maximum length of input sequences (e.g., 512)
# tokenizer: The tokenizer configured for the model
# training_arguments: The training arguments set up earlier
# packing: Whether to pack sequences to utilize maximum length efficiently (e.g., True or False)

# Explanation:
# - model=model: Specifies the pre-trained model to fine-tune.
# - train_dataset=dataset: Provides the training dataset to be used.
# - peft_config=peft_config: Supplies the configuration for Low-Rank Adaptation (LoRA), enabling efficient fine-tuning.
# - dataset_text_field="text": Indicates the field in the dataset that contains the text data for training.
# - max_seq_length=max_seq_length: Sets the maximum sequence length for the input data, ensuring all sequences are padded/truncated to this length.
# - tokenizer=tokenizer: Provides the tokenizer to preprocess the text data.
# - args=training_arguments: Passes the training arguments configuration, including settings like batch size, learning rate, and number of epochs.
# - packing=packing: Determines whether to pack sequences to maximize the use of the maximum sequence length, potentially improving training efficiency.

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [24]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.408300
50,1.658000
75,1.213800
100,1.439300
125,1.178300
150,1.360900
175,1.173100
200,1.459700
225,1.157400
250,1.533300


TrainOutput(global_step=250, training_loss=1.3582038269042969, metrics={'train_runtime': 1721.3162, 'train_samples_per_second': 0.581, 'train_steps_per_second': 0.145, 'total_flos': 8755214190673920.0, 'train_loss': 1.3582038269042969, 'epoch': 1.0})

In [25]:
trainer.model.save_pretrained(new_model)

In [ ]:
# Reload the pre-trained model in FP16 and configure it for low CPU memory usage
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,                      # The name or path of the pre-trained model
    low_cpu_mem_usage=True,          # Load the model using less CPU memory
    return_dict=True,                # Return outputs as dictionaries
    torch_dtype=torch.float16,       # Load the model in FP16 precision
    device_map=device_map,           # A mapping of model components to devices
)

# Merge the LoRA weights with the base model
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()     # Merge the LoRA weights and unload unnecessary components

# Reload the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the padding token to be the same as the end-of-sequence (eos) token
tokenizer.pad_token = tokenizer.eos_token

# Set the padding side to 'right' to fix overflow issues with FP16 training
tokenizer.padding_side = "right"

# Explanation:
# - AutoModelForCausalLM.from_pretrained(): Reloads the pre-trained model in FP16 precision with low CPU memory usage.
# - PeftModel.from_pretrained(): Merges the LoRA weights with the base model.
# - merge_and_unload(): Merges the LoRA weights into the base model and unloads unnecessary components.
# - AutoTokenizer.from_pretrained(): Reloads the tokenizer for the specified model.
# - tokenizer.pad_token = tokenizer.eos_token: Sets the padding token to be the same as the eos token.
# - tokenizer.padding_side = "right": Sets the padding side to 'right' to address overflow issues with FP16 training.

In [27]:
# Override the default preferred encoding to UTF-8
import locale

# Set the preferred encoding to UTF-8
# This can help avoid issues related to different default encodings on various systems
locale.getpreferredencoding = lambda: "UTF-8"

# Explanation:
# - import locale: Imports the locale module, which provides access to the locale-related functionality.
# - locale.getpreferredencoding = lambda: "UTF-8": Overrides the default getpreferredencoding function
#   to always return "UTF-8". This ensures that the preferred encoding is consistently set to UTF-8,
#   which can help avoid issues related to text encoding, especially when working with international data.

In [28]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
model.push_to_hub("ShehbazPatel/Llama-2-7b-chat-new-finetune")

In [32]:
tokenizer.push_to_hub("ShehbazPatel/Llama-2-7b-chat-new-finetune")

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ShehbazPatel/Llama-2-7b-chat-new-finetune/commit/1beffa791645b31654d1a72c3fabf4b4c852ce9c', commit_message='Upload tokenizer', commit_description='', oid='1beffa791645b31654d1a72c3fabf4b4c852ce9c', pr_url=None, pr_revision=None, pr_num=None)

In [33]:
# Install the transformers and accelerate packages using pip
!pip install transformers accelerate

# Explanation:
# - !pip install: This command runs the pip package installer. The exclamation mark (!) is used to execute shell commands in a Jupyter notebook.
# - transformers: This package provides state-of-the-art natural language processing (NLP) models and tools from Hugging Face.
# - accelerate: This package helps with the distributed training and inference of models, making it easier to manage multi-GPU setups and mixed precision training.

In [34]:
from transformers import AutoTokenizer
import transformers
import torch

In [42]:
model="snshrivas10/Llama-2-7b-chat-new-finetune"

In [43]:
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [37]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 687.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.


In [44]:
prompt = "What is a large language model?"

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
sequences = pipeline(
    f'<s>[INST] {prompt} [/INST]',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

In [ ]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")